# 第5部　統計的推定
## 6章　区間推定

### 実装：分析の準備

In [1]:
# 数値計算に使うライブラリ
import numpy as np
import pandas as pd
from scipy import stats

# グラフを描画するライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

# グラフの日本語表記
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = 'Meiryo'

In [2]:
# 表示設定(書籍本文のレイアウトと合わせるためであり、必須ではありません)
np.set_printoptions(linewidth=60)
pd.set_option('display.width', 60)

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 8, 4

In [3]:
# データの読み込み
fish = pd.read_csv('5-6-1-fish_length.csv')['length']
fish

0    4.352982
1    3.735304
2    5.944617
3    3.798326
4    4.087688
5    5.265985
6    3.272614
7    3.526691
8    4.150083
9    3.736104
Name: length, dtype: float64

### 実装：点推定

In [4]:
# 点推定
x_bar = np.mean(fish)
u2 = np.var(fish, ddof=1)

print('標本平均：', round(x_bar, 3))
print('不偏分散：', round(u2, 3))

標本平均： 4.187
不偏分散： 0.68


### 実装：母平均の区間推定

#### 定義通りの実装

In [5]:
# 統計量の計算
n = len(fish)            # サンプルサイズ
df = n - 1               # 自由度
u = np.std(fish, ddof=1) # 標準偏差
se = u / np.sqrt(n)      # 標準誤差

print('サンプルサイズ：', n)
print('自由度　　　　：', df)
print('標準偏差　　　：', round(u, 3))
print('標準誤差　　　：', round(se, 3))
print('標本平均　　　：', round(x_bar, 3))

サンプルサイズ： 10
自由度　　　　： 9
標準偏差　　　： 0.825
標準誤差　　　： 0.261
標本平均　　　： 4.187


In [6]:
# 2.5%点と97.5%点
t_025 = stats.t.ppf(q=0.025, df=df)
t_975 = stats.t.ppf(q=0.975, df=df)

print('t分布の 2.5%点：', round(t_025, 3))
print('t分布の97.5%点：', round(t_975, 3))

t分布の 2.5%点： -2.262
t分布の97.5%点： 2.262


In [7]:
# 母平均の区間推定
lower_mu = x_bar - t_975 * se
upper_mu = x_bar - t_025 * se

print('下側信頼限界：', round(lower_mu, 3))
print('上側信頼限界：', round(upper_mu, 3))

下側信頼限界： 3.597
上側信頼限界： 4.777


#### 簡単な実装方法

In [8]:
# 区間推定
res_1 = stats.t.interval(alpha=0.95, df=df, loc=x_bar, scale=se)
np.round(res_1, 3)

array([3.597, 4.777])

### 信頼区間の幅を決める要素

In [9]:
# 標準偏差が大きいと、信頼区間は広くなる
se_2 = (u * 10) / np.sqrt(n)
res_2 = stats.t.interval(alpha=0.95, df=df, loc=x_bar, scale=se_2)
np.round(res_2, 3)

array([-1.713, 10.087])

In [10]:
# サンプルサイズが大きいと、信頼区間は狭くなる
n_2 = n * 10
df_2 = n_2 - 1
se_3 = u / np.sqrt(n_2)
res_3 = stats.t.interval(alpha=0.95, df=df_2, loc=x_bar, scale=se_3)
np.round(res_3, 3)

array([4.023, 4.351])

In [11]:
# 99%信頼区間
res_4 = stats.t.interval(alpha=0.99, df=df, loc=x_bar, scale=se)
np.round(res_4, 3)

array([3.339, 5.035])

### 区間推定の結果の解釈

In [12]:
# 母集団分布（母平均は4）
norm_dist = stats.norm(loc=4, scale=0.8)

In [13]:
num_trials = 20000 # シミュレーションの繰り返し数
included_num = 0   # 信頼区間が母平均(4)を含んでいた回数

In [14]:
# 「データを10個選んで95%信頼区間を求める」試行を20000回繰り返す
np.random.seed(1) # 乱数の種
for i in range(0, num_trials):
    # 標本の抽出
    sample = norm_dist.rvs(size=n)
    # 信頼区間の計算
    df = n - 1                 # 自由度
    x_bar = np.mean(sample)    # 標本平均
    u = np.std(sample, ddof=1) # 標準偏差
    se = u / np.sqrt(n)        # 標準誤差
    interval = stats.t.interval(0.95, df, x_bar, se)
    # 信頼区間が母平均(4)を含んでいた回数をカウント
    if(interval[0] <= 4 <= interval[1]):
        included_num = included_num + 1

In [15]:
# 全試行中、信頼区間が母平均（4）を含んでいた割合
included_num / num_trials

0.948

### 実装：母分散の区間推定

In [16]:
# 2.5%点と97.5%点
chi2_025 = stats.chi2.ppf(q=0.025, df=df)
chi2_975 = stats.chi2.ppf(q=0.975, df=df)

print('χ2分布の 2.5%点：', round(chi2_025, 3))
print('χ2分布の97.5%点：', round(chi2_975, 3))

χ2分布の 2.5%点： 2.7
χ2分布の97.5%点： 19.023


In [17]:
# 母分散の区間推定
upper_sigma = (n - 1) * u2 / chi2_025
lower_sigma = (n - 1) * u2 / chi2_975

print('下側信頼限界：', round(lower_sigma, 3))
print('上側信頼限界：', round(upper_sigma, 3))

下側信頼限界： 0.322
上側信頼限界： 2.267
